In [ ]:
#import all necessary libraries
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
import random

In [ ]:
# crawl food names and ingredients
food_list = []

ellipsis_pattern = re.compile(r'\.{2,}')

def remove_ellipsis(ingredient):
    return re.sub(ellipsis_pattern, '', ingredient)

for i in range(1,15):
    url = f"https://www.russianfood.com/recipes/bytype/?fid=123&page={i}#rcp_list"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "lxml")

    food_information = soup.find_all("div", class_="recipe_l in_seen v2")

    for food in food_information:

        #food_name_element = food.find('span', itemprop='name')
        food_name_element = food.find('a', href=True, attrs={"name": re.compile("^el\d+")})
        food_name = food_name_element.text if food_name_element else 'Unknown'




        ingredients_element = food.find('div', class_="ingr_str")
        ingredients_str = ingredients_element.text if ingredients_element else ''
        ingredients_str = ingredients_str.strip('\xa0\n')
        ingredients_list = [ingredient.strip() for ingredient in ingredients_str.split(',')]
        ingredients_list = [remove_ellipsis(ingredient) for ingredient in ingredients_list]

        #link_tag = food.find('a', itemprop='url').text
        link_tag =  food.find('a')
        link = link_tag["href"]

        food_list.append([food_name, ingredients_list, link])

In [ ]:
#define user_agents to handle webscraping errors

user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
]



In [ ]:
# transform the list to dataframe and save it
df = pd.DataFrame(food_list,columns = ["Name","Ingredients","link to dish"])

#optionally save it
#df.to_csv("food.csv")

In [ ]:


def get_recipe_steps(link):
    try:
        base_url = "https://www.russianfood.com"
        response = requests.get(base_url + link,headers={'User-Agent': random.choice(user_agents_list)})

        # Check the status code
        if response.status_code != 200:
            print(f"Failed to retrieve {base_url + link}. Status code: {response.status_code}")
            return []

        soup = BeautifulSoup(response.content, "lxml")


        # First check the common location
        recipe_divs = soup.find_all("div", class_="step_n")

        steps = []
        for step in recipe_divs:
            step_text_element = step.find('p')
            if step_text_element:  # Ensure there's a paragraph element to extract
                step_text = step_text_element.text.replace('\r', '').replace('\n', '')
                steps.append(step_text)

        # If no steps found in the common location, check the alternative location
        if not steps:
            alternative_div = soup.find("div", id="how")
            if alternative_div:
                for paragraph in alternative_div.find_all('p'):
                    step_text = paragraph.text.replace('\r', '').replace('\n', '')
                    steps.append(step_text)

    except NoSuchWindowException:
            print("The target window is closed or not accessible.")
            steps = []  # Return an empty list or handle in any other way

    except InvalidSessionIdException:
            print("The WebDriver session is invalid. Maybe the browser was closed.")
            steps = []  # Return an empty list or handle in any other way

    return steps



steps_list = []

for _, _, link in food_list:
    steps_list.append(get_recipe_steps(link))
    time.sleep(4)


In [ ]:
# THERE COULD BE AN ERROR DUE TO CRAWLING LIMITS , THEN USE THIS CODE TO CRAWL DATA PARTIALLY :

#for _, _, link in food_list[100:151]:
   # steps_list.append(get_recipe_steps(link))
   # time.sleep(4)
#df = pd.DataFrame(food_list[50:101],columns = ["Name","Ingredients","link to dish"])

In [ ]:
# Save crawled names, ingredients and recipe instructions to one dataframe
df = pd.DataFrame(food_list,columns = ["Name","Ingredients","link to dish"])
df['Making steps'] = steps_list

In [ ]:
# Save the file
df.to_csv("final_uzb_recipe.csv")